# Conector MT5 Python

In [239]:
import pandas as pd
import MetaTrader5 as mt5

In [240]:
# Creamos un diccionario con las credenciales.

creds = {
    'path':'C:\\Program Files\\Darwinex MetaTrader 5\\terminal64.exe',
    ## MetaQuotes Account
    'login': 84666107,
    'pss':'D@D7NvSa',
    'server':'MetaQuotes-Demo',

    ## Darwinex Account
    # 'login' : 4000040131,
    # 'pss':'$iC4kd$8%K0',
    # 'server':'liveUK-mt5.darwinex.com',
    
    'timeout':60000,
    'portable':False,
}

In [241]:
creds['path']

'C:\\Program Files\\Darwinex MetaTrader 5\\terminal64.exe'

In [242]:
# Conexion a MT5
if mt5.initialize(path=creds['path'], login=creds['login'], password=creds['pss'], server=creds['server'],
        timeout=creds['timeout'], portable=creds['portable']):
    print('MT5 lanzado correctamente y conectado.')
else:
    print(f'Ha habido un problema iniciando: {mt5.last_error()}')

MT5 lanzado correctamente y conectado.


# Informacion del Terminal

In [243]:
# Solicitamos la info de MT5 pero no podemos modificarla desde Python
terminal_info_dic = mt5.terminal_info()._asdict()

In [244]:
# Lo metemos en un dataframe.
terminal_info_df = pd.DataFrame(terminal_info_dic, index=['Set'])

In [245]:
# Imprimimos el datadrame.
terminal_info_df

,community_account,community_connection,connected,dlls_allowed,trade_allowed,tradeapi_disabled,email_enabled,ftp_enabled,notifications_enabled,mqid,...,codepage,ping_last,community_balance,retransmission,company,name,language,path,data_path,commondata_path
Set,False,False,True,True,True,False,False,False,False,False,...,1252,45809,0.0,0.0,Tradeslide Trading Tech Limited,Darwinex MetaTrader 5,Spanish,C:\Program Files\Darwinex MetaTrader 5,C:\Users\joalr\AppData\Roaming\MetaQuotes\Term...,C:\Users\joalr\AppData\Roaming\MetaQuotes\Term...


# Informacion de la cuenta

In [246]:
# Aqui almacenamos la informacion de la cuenta, en un diccionario.
account_info_dict = mt5.account_info()._asdict()

# Asi al momento de necesitar saber algun dato especifico lo tendremos ordenado y segmentado.
account_info_dict['profit']
account_info_dict['balance']


100000.0

# Recuperacion de datos: Symbols

In [247]:
# Para ver la cantidad de simbolos.
mt5.symbols_total()

140

In [248]:
# Para filtrar simbolos.
mt5.symbols_get() # Devolvera todos los simbolos en una lista fea de leer.
mt5.symbols_get('*EUR*') # Devolvera todos los simbolos relacionados con el ACTIVO SELECCIONADO en una lista fea de leer.
symbols = mt5.symbols_get('*EUR*') # Los metemos en una variable.

In [249]:
# Con un bucle for recorremos solo los nombres, es mucho mejor para el ojo.
for symbol in symbols:
    print(symbol.name)

EURUSD
EURAUD
EURCAD
EURCHF
EURCZK
EURDKK
EURGBP
EURHKD
EURHUF
EURJPY
EURNOK
EURNZD
EURPLN
EURRUR
EURRUB
EURSEK
EURTRY
EURZAR
XAUEUR
XAGEUR
EURMXN
EURSGD
EURILS
EURCNH


In [250]:
# Acceder a la informacion de un simbolo en concreto.
mt5.symbol_info('EURUSD')
eurusd = mt5.symbol_info('EURUSD')._asdict() # Lo metemos como un diccionario en una variable.

In [251]:
# Igual que antes podemos acceder siempre a la informacion especifica del diccionario.
eurusd['digits']

5

In [252]:
# Lo metemos PLANAMENTE en una variable.
eurjpy = mt5.symbol_info('EURJPY')

In [253]:
# De esta forma tambien podriamos acceder, atravez de la nomenclatura del punto.
eurjpy.digits

3

##### Agregar al Market Watch

In [254]:
#Agregamos un simbolo al market watch, que creo, sera necesario para poder operarlo desde python.
if (mt5.symbol_select('EURUSD',True)):
    print('El simbolo ha sido agregado.')
else:
    print(f'Error al agregar el simbolo: {mt5.last_error()}')

El simbolo ha sido agregado.


##### InfoTICK sobre un activo.

In [255]:
#Pedimos informacion sobre el ultimo tick de un activo x.
symbol_info = mt5.symbol_info_tick('EURUSD')
symbol_info.ask

1.07837

In [256]:
mt5.symbol_info_tick('EURUSD')._asdict()

{'time': 1722541827,
 'bid': 1.07836,
 'ask': 1.07837,
 'last': 0.0,
 'volume': 0,
 'time_msc': 1722541827686,
 'flags': 1028,
 'volume_real': 0.0}

# Adquisicion de datos OHLC
 
###### Importaremos datos de calidad con formato OHLC que es como estamos acostumbrados a verlos. Veremos la importancia de trabajar con las zonas horarias correctas.

In [257]:

from datetime import datetime
import pytz

In [258]:
symbol = 'EURNOK'
timeframe = mt5.TIMEFRAME_H1

# Recuperamos datos desde una fecha determinada
date = datetime.strptime('2023-07-27 15:00:00', '%Y-%m-%d %H:%M:%S')
date

datetime.datetime(2023, 7, 27, 15, 0)

In [259]:
# NO LO USAREMOS.
# Convertir mi zona horaria a UTC
# date.astimezone(pytz.utc)
# Esto para calcular la hora que le he dado a la hora que corresponde en UTC (ES UN DOLOR DE CABEZA)

In [260]:
# Indicar que mi zona horaria ya esta en UTC
date.replace(tzinfo=pytz.utc)
date = date.replace(tzinfo=pytz.utc)
# De esta forma directamente le indicaremos la hora en UTC, es decir no tenemos que calcular los cambios de horario.

In [261]:
# Llamamos 10 datos de tiempo desde la fecha y hora establecida. (Por eso el 10, al final)
raw_ohlc_data = mt5.copy_rates_from(symbol, timeframe, date, 10)
# Lo metemos en un DF
df = pd.DataFrame(raw_ohlc_data)
# Y pasamos finalmente la columna time del DF a formato datetime, indicando que la unidad es de segundos.
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)

In [262]:
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2023-07-27 06:00:00,11.17320,11.17430,11.16679,11.16680,678,102,0
2023-07-27 07:00:00,11.16650,11.16810,11.16253,11.16719,1128,102,0
2023-07-27 08:00:00,11.16739,11.18670,11.16457,11.17829,2507,102,0
2023-07-27 09:00:00,11.17989,11.19951,11.17759,11.19951,3280,102,0
2023-07-27 10:00:00,11.19759,11.19945,11.14200,11.14894,1787,102,0
2023-07-27 11:00:00,11.14869,11.16669,11.13309,11.14469,2076,102,0
2023-07-27 12:00:00,11.14454,11.17724,11.13354,11.17424,2273,102,0
2023-07-27 13:00:00,11.17429,11.19354,11.15510,11.16639,2072,102,0
2023-07-27 14:00:00,11.16634,11.17171,11.11764,11.12069,2189,102,0


In [263]:
# Llamamos 10 datos de tiempo desde la ultima vela, hacia el numero de velas anterior que le digamos.
raw_ohlc_data = mt5.copy_rates_from_pos(symbol, timeframe, 1, 10)
# Lo metemos en un DF
df = pd.DataFrame(raw_ohlc_data)
# Y pasamos finalmente la columna time del DF a formato datetime, indicando que la unidad es de segundos.
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)


# Tener en cuenta que los datos van a estar en la timezone Asia/Nicosia y en DST de US.

In [264]:
df

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2024-08-01 09:00:00,11.80558,11.80831,11.78126,11.79485,3142,282,0
2024-08-01 10:00:00,11.79463,11.79487,11.74969,11.76876,3589,282,0
2024-08-01 11:00:00,11.76670,11.76940,11.74440,11.75081,3202,282,0
2024-08-01 12:00:00,11.75077,11.75409,11.72443,11.74893,2631,185,0
2024-08-01 13:00:00,11.74911,11.76617,11.73710,11.73740,2572,282,0
2024-08-01 14:00:00,11.73739,11.76721,11.73724,11.75099,2877,282,0
2024-08-01 15:00:00,11.75101,11.75494,11.72764,11.74085,3338,103,0
2024-08-01 16:00:00,11.74060,11.75281,11.73159,11.73914,3334,282,0
2024-08-01 17:00:00,11.73787,11.78174,11.73572,11.75184,4622,282,0


In [265]:

#Recuperar datos entre dos fechas

date_ini = datetime.strptime('2023-07-13 15:00:00', '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.utc)
date_end = datetime.strptime('2023-07-17 15:00:00', '%Y-%m-%d %H:%M:%S').replace(tzinfo=pytz.utc)
raw_ohlc_data = mt5.copy_rates_range(symbol, timeframe, date_ini, date_end)
df = pd.DataFrame(raw_ohlc_data)
df['time'] = pd.to_datetime(df['time'], unit='s')
df.set_index('time', inplace=True)


In [266]:
df.tail()

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2023-07-17 11:00:00,11.24809,11.29360,11.24781,11.29130,2391,102,0
2023-07-17 12:00:00,11.29140,11.31569,11.25354,11.29189,2347,102,0
2023-07-17 13:00:00,11.29185,11.29630,11.26695,11.27417,1986,102,0
2023-07-17 14:00:00,11.27410,11.27435,11.24279,11.24324,1810,102,0
2023-07-17 15:00:00,11.24324,11.26760,11.23560,11.26760,2640,102,0


# Adquisicion de datos en TICK

In [267]:
symbol = 'EURUSD'

# Para embellezer el codigo:
formato = '%Y-%m-%d %H:%M:%S'

# Definimos la fecha de inicio y de final
date_ini = datetime.strptime('2023-07-13 15:00:00', formato).replace(tzinfo=pytz.utc)
date_end = datetime.strptime('2023-07-17 15:00:00', formato).replace(tzinfo=pytz.utc)

In [268]:
# Recogemos desde los proximos 5000 ticks desde la fecha de inicio indicada.
df = pd.DataFrame(mt5.copy_ticks_from(symbol, date_ini, 5000, mt5.COPY_TICKS_ALL))
df['time_msc'] = pd.to_datetime(df['time_msc'], unit='ms')
df.set_index('time_msc', inplace=True)

In [269]:
df['spread'] = df['ask'] - df['bid']
df.head()

,time,bid,ask,last,volume,flags,volume_real,spread
time_msc,,,,,,,,
2023-07-13 15:00:00.022,1689260400,1.11745,1.11745,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.223,1689260400,1.11746,1.11746,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.571,1689260400,1.11748,1.11748,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.667,1689260400,1.11749,1.11749,0.0,0,134,0.0,0.0
2023-07-13 15:00:00.741,1689260400,1.11750,1.11750,0.0,0,134,0.0,0.0


In [270]:
df_2 = pd.DataFrame(mt5.copy_ticks_range(symbol, date_ini, date_end, mt5.COPY_TICKS_ALL))
df_2['time_msc'] = pd.to_datetime(df_2['time_msc'], unit='ms')
df_2.set_index('time_msc', inplace=True)

In [271]:
df_2

,time,bid,ask,last,volume,flags,volume_real
time_msc,,,,,,,
2023-07-13 15:00:00.022,1689260400,1.11745,1.11745,0.0,0,134,0.0
2023-07-13 15:00:00.223,1689260400,1.11746,1.11746,0.0,0,134,0.0
2023-07-13 15:00:00.571,1689260400,1.11748,1.11748,0.0,0,134,0.0
2023-07-13 15:00:00.667,1689260400,1.11749,1.11749,0.0,0,134,0.0
2023-07-13 15:00:00.741,1689260400,1.11750,1.11750,0.0,0,134,0.0
...,...,...,...,...,...,...,...
2023-07-17 14:59:57.576,1689605997,1.12342,1.12342,0.0,0,134,0.0
2023-07-17 14:59:59.209,1689605999,1.12341,1.12341,0.0,0,134,0.0
2023-07-17 14:59:59.229,1689605999,1.12342,1.12342,0.0,0,134,0.0


# Envio y Cierre de Ordenes

In [354]:
# Primero, definimos el símbolo en el que queremos abrir nuestras operaciones
symbol = 'USDSEK'

# Y nos aseguramos de que dicho símbolo esté añadido al MarketWatch
if mt5.symbol_select(symbol, True):
    print("Símbolo añadido correctamente")
else:
    print(f"Ha ocurrido un error añadiendo el símbolo: {mt5.last_error()}")

Símbolo añadido correctamente


In [355]:
# #Creacion del trade Request.
# request = {
#     'action': mt5.TRADE_ACTION_DEAL, #Enviamos una orden a mercado.
#     'magic': 1010101010, #Magic number es el id de mi estrategia, debe tener 10 numeros. 
#     #'order'        : para manejar ordenes pendientes... NO ES EL CASO.
#     'symbol': symbol,
#     'volume': 1.25, #Un numero entero siempre positivo.
#     #'price'        : Siendo una orden a mercado no necesitamos este valor.
#     #'stoplimit'    : Siendo una orden a mercado no necesitamos este valor.
#     'sl': 0.0, #StopLoss
#     'tp': 0.0, #TakeProfit
#     'deviation': 50,   #Slippage que estamos dispuestos a tolerar.
#     'type': mt5.ORDER_TYPE_BUY, #Tipo de orden (Compra, venta, limits...)
#     'type_filling': mt5.ORDER_FILLING_FOK, #Forma de relleno de la orden, ver mas a detalle, en este caso fill or kill.
#     'type_time': mt5.ORDER_TIME_GTC, #Si queremos cerrarla con condicion de tiempo, en este caso Got till Cancel es para tenerla abierta hasta que la cerremos nosotros.
#     'comment': "Mi primera orden desde python."
# } 
# request  

# POR ALGUNA EXTRANIA RAZON NO SE EJECUTA BIEN CON LOS COMENTARIOS Y LAS TABULACIONES.

In [361]:

# Creamos el trade request necesario
request = {
    'action': mt5.TRADE_ACTION_DEAL,
    'magic': 1010101010,
    'symbol': symbol,
    'volume': 1.25,
    'sl': 10.73,
    'tp': 10.75,
    # 'price': mt5.symbol_info_tick(symbol).ask,
    'deviation': 50,
    'type': mt5.ORDER_TYPE_BUY,
    'type_filling': mt5.ORDER_FILLING_FOK,
    'type_time': mt5.ORDER_TIME_GTC,
    'comment': "Primera orden desde Python"
}

In [362]:
# Y enviamos el request para poder ejecutar la orden
order = mt5.order_send(request)
print(order)

if order.retcode == mt5.TRADE_RETCODE_DONE:
    print(f"La orden se ha ejecutado correctamente: {order}")
else:
    print(f"Ha ocurrido un error al ejecutar la orden: {mt5.last_error()}")

OrderSendResult(retcode=10009, deal=2678937740, order=2704612315, volume=1.25, price=10.7341, bid=0.0, ask=0.0, comment='Request executed', request_id=2965612118, retcode_external=0, request=TradeRequest(action=1, magic=1010101010, order=0, symbol='USDSEK', volume=1.25, price=0.0, stoplimit=0.0, sl=10.73, tp=10.75, deviation=50, type=0, type_filling=0, type_time=0, expiration=0, comment='Primera orden desde Python', position=0, position_by=0))
La orden se ha ejecutado correctamente: OrderSendResult(retcode=10009, deal=2678937740, order=2704612315, volume=1.25, price=10.7341, bid=0.0, ask=0.0, comment='Request executed', request_id=2965612118, retcode_external=0, request=TradeRequest(action=1, magic=1010101010, order=0, symbol='USDSEK', volume=1.25, price=0.0, stoplimit=0.0, sl=10.73, tp=10.75, deviation=50, type=0, type_filling=0, type_time=0, expiration=0, comment='Primera orden desde Python', position=0, position_by=0))
